In [1]:
# Wavefunction generation
from pyscf import mcscf, fci, lib 
from pyscf import gto, scf, tools, dft
import api as pyq 
import importlib
import os
import h5py
import pandas as pd
import pdb
import pyscf
import numpy as np
import pyqmc
import scipy
import copy
import gpu
import matplotlib.pyplot as plt
from pyscf.scf.hf import dump_scf_summary
from concurrent.futures import ProcessPoolExecutor
from pyscf.scf.chkfile import dump_scf
print(pyq.__file__)
print(pyscf.__file__)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/__init__.py


In [2]:
# Wavefunction generation
from scf_runs import run_lda_he, run_casci
scf_checkfile, opt_checkfile, mf_lda = run_lda_he()
ci_checkfile, mc, opt_chkfile, vmc_chkfile = run_casci(scf_checkfile, nroots=1, ncas = 2)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/Users/ksu/Software/anaconda3/lib/python3.11/site-packages/pyscf/__init__.py
He atom neutral LDA spin=0
HF
converged SCF energy = -2.86151422722833  <S^2> = 2.4424907e-15  2S+1 = 1
LDA
converged SCF energy = -2.83456474290722  <S^2> = 3.1086245e-15  2S+1 = 1
CASCI nelecas up/down (1, 1)
CASCI E = -2.86531899726653  E(CI) = -2.86531899726653  S^2 = 0.0000000
Available output from CASCI: <KeysViewHDF5 ['ci', 'fci', 'mc_mo_energy', 'mf_mo_energy', 'mo_coeff', 'mo_occ', 'ncas', 'ncore', 'nelecas']>


In [31]:
import importlib, recipes, bosonrecipes, bosonslater, bosonaccumulators, bosonwftools, bosonslater_wf_based
modules = [recipes, bosonrecipes, bosonrecipes, bosonslater, bosonaccumulators, bosonwftools] # Reloads recipes.py file after each update
for m in modules:
    importlib.reload(m)


wf = bosonslater.BosonWF(mf_lda.mol, mf_lda, mc)
wf_old = bosonslater_wf_based.BosonWF(mf_lda.mol, mf_lda, mc)
configs = bosonrecipes.initial_guess(mf_lda.mol, 1)

In [32]:
configs.configs.shape

(1, 2, 3)

In [35]:
tol = 1E-1

nconf, nelec, _ = configs.configs.shape

def is_equal(a, b, tag=None):
    result = (np.abs(a-b) < tol).all()
    if not result:
        print(tag, result, 'diff', np.max(np.abs(a-b)), 'a', np.max(np.abs(a)), 'b', np.max(np.abs(b)))
    else:
        print(tag, result)
    return result
    
def limdrift(g, cutoff=1):
    """
    Limit a vector to have a maximum magnitude of cutoff while maintaining direction

    :parameter g: a [nconf,ndim] vector
    :parameter cutoff: the maximum magnitude
    :returns: The vector with the cutoff applied.
    """
    tot = np.linalg.norm(g, axis=1)
    mask = tot > cutoff
    g[mask, :] = cutoff * g[mask, :] / tot[mask, np.newaxis]
    return g

def gradient_value_wf_based(wf_old, e, epos):
    s = int(e >= wf_old._nelec[0])
    aograd = wf_old.orbitals.aos("GTOval_sph_deriv1", epos)
    mograd = wf_old.orbitals.mos(aograd, s)

    grads = []
    ratios = []
    num_wf = len(wf_old.wfs)
    ci = 1./num_wf        
    for wf in wf_old.wfs: # over n 
        wf_grad, wf_ratio, _ = wf.gradient_value(e, epos)
        grads.append(wf_grad)
        ratios.append(wf_ratio)
    grads = gpu.asnumpy(np.array(grads))

    value, real_values, _ = wf_old.value_all()
    grad = ci * np.einsum('di, dgi, i->gi', real_values, grads, 1./np.exp(2*value))
    
    # print('wf_based real_values', ci * real_values)
    # print('wf_based val no sq', np.exp(value))
    # print('wf_based grads', grads)
    # print()
    # print(real_values[0][0:10])
    # print(value[0:10])
    # print()
    ratios = gpu.asnumpy(np.array(ratios))
    # import pdb
    # pdb.set_trace()
    # ratiosf = np.ones(ratios.shape[1])
    ratiosf = np.mean(ratios, axis=0)

    # ratiosf = np.sqrt(np.sum(ratios**2, axis=0))
    # ratiosf = np.average(ratios, weights=1./real_values, axis=0)
    # ratiosf = np.sqrt(np.average(ratios, weights=real_values, axis=0))/np.exp(value)
    # ratiosf = np.average(ratios, weights=real_values, axis=0)/np.exp(2*value)
    # import pdb
    # pdb.set_trace()
    # ratiosf = np.ones(ratiosf.shape)/2
    # debug = (num_wf, grads, real_values, value)
    debug = grads
    return grad, ratiosf, (aograd[:, 0], mograd[0]), debug
    
def gradient_value_regular(wf, e, epos):   
    s = int(e >= wf._nelec[0])
    aograd = wf.orbitals.aos("GTOval_sph_deriv1", epos)
    mograd = wf.orbitals.mos(aograd, s)
    mograd_vals = mograd[:, :, wf._det_occup[s]]
    import pdb
    
    jacobi = gpu.cp.einsum(
        "ei...dj,idj...->ei...d",
        mograd_vals,
        wf._inverse[s][..., e - s * wf._nelec[0]],
    )
    # pdb.set_trace()
    det_coeff = wf.myparameters['det_coeff']
    upref = gpu.cp.amax(wf._dets[0][1]).real
    dnref = gpu.cp.amax(wf._dets[1][1]).real

    det_array = (
        wf._dets[0][0, :, wf._det_map[0]]
        * wf._dets[1][0, :, wf._det_map[1]]
        * gpu.cp.exp(
            wf._dets[0][1, :, wf._det_map[0]]
            + wf._dets[1][1, :, wf._det_map[1]]
            - upref
            - dnref
        )
    )
    # print()
    # print(det_array[0][0:10])
    # print()
    jacobid = jacobi[..., wf._det_map[s]]
    jacobid = jacobid[1:]/jacobid[0]
    # import pdb
    # pdb.set_trace()
    numer =  gpu.cp.einsum(
        "ei...d,d,di->ei...",
        jacobid,
        det_coeff,
        det_array**2
    )

    numer_ratio =  gpu.cp.einsum(
        "i...d,d,di->i...",
        jacobid[0]**2,
        det_coeff,
        det_array**2
    )    
    # numerd =  gpu.cp.einsum(
    #     "ei...d,d,di->dei...",
    #     jacobi[..., wf._det_map[s]],
    #     det_coeff,
    #     det_array**2
    # )[:, 1:, :]
    
    denom = gpu.cp.einsum(
        "d,di->i...",
        det_coeff,
        det_array**2
    )

    # denomd = gpu.cp.einsum(
    #     "d,di->di...",
    #     det_coeff,
    #     det_array**2
    # )
    # print(det_coeff)
    # val = np.sqrt(denom)  * np.exp((upref + dnref))
    # real_values = denomd  * np.exp(2*(upref + dnref))
    # print('reg real_values', real_values)
    # print('reg val np sq', val)
    # print('reg jacobi', jacobid)
    # print(val-np.exp(wf.value()[1]))
    
    ratios = numer / denom
    derivatives = ratios #ratios[1:] #/ ratios[0]
    derivatives[~np.isfinite(derivatives)] = 0.0
    values = numer_ratio/denom
    values[~np.isfinite(values)] = 1.0
    # debug = (det_coeff, det_array, values, numerd, denomd)
    debug = jacobi[..., wf._det_map[s]][1:]
    return derivatives, values, (aograd[:, 0], mograd[0]), debug


In [36]:
nsteps = 10
nelec = 2
tstep = 0.3
for _ in range(nsteps):
    acc = 0.0
    for e in range(nelec):
        import pdb
        sign_p,  val_p   = wf.recompute(configs)
        # sign_p2, val_p2  = wf_old.recompute(configs)
        # is_equal(val_p, val_p2, 'val')
        
        wf_k = copy.deepcopy(wf)
        # wf_old_k = copy.deepcopy(wf_old)

        # g, _, _ = wf.gradient_value(e, configs.electron(e))
        # g_old, _, _ = wf_old.gradient_value(e, configs.electron(e))
        
        g, _, _, _ = gradient_value_regular(wf, e, configs.electron(e))
        # g_old, _, _, _ = gradient_value_wf_based(wf_old, e, configs.electron(e))
        
        # is_equal(g, g_old, 'g')
        grad = limdrift(np.real(g.T)) 
        # grad_old = limdrift(np.real(g_old.T)) 
        # is_equal(g, g_old, 'g')
        gauss = np.random.normal(scale=np.sqrt(tstep), size=(nconf, 3))

        # is_equal(grad, grad_old, 'grad')
        newcoorde = configs.configs[:, e, :] + gauss + grad * tstep
        newcoorde = configs.make_irreducible(e, newcoorde)

        # # Compute reverse move
        g_k, new_val, saved, dbg = gradient_value_regular(wf_k, e, newcoorde)
        # g_k_wfb, _, saved_old, debug_wfb = gradient_value_wf_based(wf_old_k, e, newcoorde)
        # is_equal(g_k, g_k_wfb, 'g_k')
        newcoord = copy.deepcopy(configs)
        newcoord.configs[:,e,:] = newcoorde.configs
        _, val = wf_k.recompute(newcoord)
        # _, val_old = wf_old_k.recompute(newcoord)
        
        ratio = np.exp(2*(val-val_p))
        ratio2 = new_val**2
        print(ratio2/ratio)
        
        


[0.00990565]
[2.36095523]
[0.06555145]
[0.00469406]
[0.00216746]
[1.56803064]
[0.00307141]
[0.47122957]
[0.02441745]
[0.20865727]
[0.47071806]
[0.01425214]
[4.74583596e-05]
[2.17360758]
[140848.40881428]
[0.21264418]
[1.14840671]
[2.33769655]
[0.0176914]
[0.28245243]
